In [1]:
import numpy as np
import pandas as pd

import os
import sys
sys.path.insert(0, '..')
import ccal

%matplotlib inline
# %config InlineBackend.figure_formats = {'svg',}

SyntaxError: invalid syntax (onco_gps.py, line 7)

# Get states

In [ ]:
h = ccal.support.read_gct('/home/cyborg/hematopoietic_cancer/result/ccle_hema_r/k9/ccle_hema_k_9_H.gct')
ks = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 25, 30]
n_clusterings = 10
filepath_prefix = 'output/ccle_hema_k_9_H_{}clustering'.format(n_clusterings)

labels, memberships, cophenetic_correlation = ccal.analyze.define_states(h, ks, n_clusterings=n_clusterings,
                                                                             filepath_prefix=filepath_prefix)

# Make Onco-GPS

In [ ]:
labels = pd.read_csv(filepath_prefix + '_labels.txt', index_col=0, sep='\t')
memberships = ccal.support.read_gct(filepath_prefix + '_memberships.gct')
annotations = ccal.support.make_random_features(1, memberships.shape[1])

In [ ]:
for n_components in range(3, min(h.shape[0], 21), 3):
    #h = ccal.support.make_random_features(n_components, 100)
    h = h.iloc[:n_components, :]
    for state in range(1, 21, 5):
        for annotation_type in ['continuous', 'categorical', 'binary']:
            ccal.onco_gps.make_map(h,
                                   np.random.choice(list(range(state)), h.shape[1]),
                                   annotations=np.random.choice(range(2), h.shape[1]),
                                   annotation_type=annotation_type)#,
                                   #background_markersize=0, background_mask_markersize=0)